In [7]:
import sqlalchemy
from sqlalchemy import create_engine, text
from tabulate import tabulate

In [14]:
DB_USER = "myuser"
DB_PASS = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydb"

In [15]:
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

In [16]:
engine = create_engine(DATABASE_URL)

## Window Functions in SQL

Window functions (also sometimes called analytic functions) are a special class of SQL functions that perform calculations across a *set of rows* that are related to the *current row*.

Unlike aggregate functions (like `SUM()`, `AVG()`, `COUNT()`), window functions *do not group rows* into a single summary row. Instead, they return a value for *each row* in the result set, based on a window of related rows.

**Essential Components:**

*   **Window Functions:** The specific function that performs the calculation (e.g., `ROW_NUMBER()`, `RANK()`, `SUM()`, `AVG()`, `LAG()`, `LEAD()`, etc.).

*   **`OVER()` Clause:**  This is *mandatory* for all window functions. It defines the "window" over which the function operates. The `OVER()` clause can contain the following:

    *   **`PARTITION BY` Clause (Optional):** Divides the rows into partitions (groups) based on one or more columns. The window function is applied *independently* to each partition. This means the calculations are performed separately for each group.

    *   **`ORDER BY` Clause (Often Required):** Specifies the order of rows *within* each partition.  Many window functions (especially those related to ranking, or accessing previous/next rows) require an `ORDER BY` clause to define the order in which the calculations are performed.

In [17]:
def execute_sql(sql_statement):
    try:
        with engine.connect() as connection:
            connection.execute(text(sql_statement))
            connection.commit()
            print(f"SQL statement executed successfully:\n{sql_statement}")
    except sqlalchemy.exc.ProgrammingError as e:
        print(f"Error executing SQL statement:\n{sql_statement}\nError: {e}")
    except Exception as e:
        print(f"An unexpected error occurred:\n{sql_statement}\nError: {e}")

In [18]:
execute_sql("""
    CREATE TABLE Products (
        product_id SERIAL PRIMARY KEY,
        product_name VARCHAR(255),
        price DECIMAL(10, 2),
        category VARCHAR(255)
    );
""")

SQL statement executed successfully:

    CREATE TABLE Products (
        product_id SERIAL PRIMARY KEY,
        product_name VARCHAR(255),
        price DECIMAL(10, 2),
        category VARCHAR(255)
    );



In [27]:
try:
    with engine.connect() as connection:
        
        connection.execute(text("TRUNCATE TABLE Products"))
        
        connection.execute(text(
                """
                   INSERT INTO Products (product_name, price, category) VALUES
                    ('Laptop', 1200.00, 'Electronics'),
                    ('Smartphone', 800.00, 'Electronics'),
                    ('Tablet', 300.00, 'Electronics'),
                    ('Book', 25.00, 'Books'),
                    ('Notebook', 10.00, 'Books'),
                    ('Pen', 5.00, 'Stationery'),
                    ('Pencil', 3.00, 'Stationery'); 
                """
        ))
        
        connection.commit()

except Exception as e:
    print("Error querying table:", e)

## 1. ROW_NUMBER()

**Definition**: Assigns a unique sequential integer to each row within a partition, based on the specified ORDER BY clause.

**Use Case**: Assigning a rank to rows based on a specific criteria.

In [28]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text(
                """
                    SELECT
                        product_name,
                        price,
                        category,
                        ROW_NUMBER() OVER (PARTITION BY category ORDER BY price DESC) AS row_num
                    FROM Products; 
                """
        ))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)

except Exception as e:
    print("Error querying table:", e)

+----------------+---------+-------------+-----------+
| product_name   |   price | category    |   row_num |
+================+=========+=============+===========+
| Book           |      25 | Books       |         1 |
+----------------+---------+-------------+-----------+
| Notebook       |      10 | Books       |         2 |
+----------------+---------+-------------+-----------+
| Laptop         |    1200 | Electronics |         1 |
+----------------+---------+-------------+-----------+
| Smartphone     |     800 | Electronics |         2 |
+----------------+---------+-------------+-----------+
| Tablet         |     300 | Electronics |         3 |
+----------------+---------+-------------+-----------+
| Pen            |       5 | Stationery  |         1 |
+----------------+---------+-------------+-----------+
| Pencil         |       3 | Stationery  |         2 |
+----------------+---------+-------------+-----------+


## 2. Ranking Functions: RANK() and DENSE_RANK()

**Definition:** Assign ranks to rows within a partition based on the specified `ORDER BY` clause.

**`RANK()`:**

*   Assigns the same rank to rows with equal values in the `ORDER BY` columns.
*   This results in gaps in the ranking sequence. For example, if two rows are tied for rank 2, the next rank assigned will be 4.

**`DENSE_RANK()`:**

*   Assigns consecutive ranks without gaps, even for rows with equal values in the `ORDER BY` columns.
*   If two rows are tied for rank 2, the next rank assigned will be 3.

**Key Difference Summarized:**

| Feature        | `RANK()`                      | `DENSE_RANK()`                |
| -------------- | ----------------------------- | ----------------------------- |
| Tie Handling | Assigns same rank, skips ranks | Assigns same rank, no skips |
| Resulting Rank | 1, 2, 2, 4, 5               | 1, 2, 2, 3, 4               |

**Use Case:**

*   Determining the top N items in a category (e.g., top 10 customers by sales, best-selling products).

**Example:**

Let's say you have a table `products` with columns `product_name` and `price`.

```sql
SELECT
    product_name,
    price,
    RANK() OVER (ORDER BY price DESC) AS product_rank,
    DENSE_RANK() OVER (ORDER BY price DESC) AS dense_rank
FROM
    products;

In [29]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text(
                """
                SELECT
                    product_name,
                    price,
                    category,
                    RANK() OVER (PARTITION BY category ORDER BY price DESC) AS rank_num,
                    DENSE_RANK() OVER (PARTITION BY category ORDER BY price DESC) AS dense_rank_num
                FROM Products;
                """
        ))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)

except Exception as e:
    print("Error querying table:", e)

+----------------+---------+-------------+------------+------------------+
| product_name   |   price | category    |   rank_num |   dense_rank_num |
+================+=========+=============+============+==================+
| Book           |      25 | Books       |          1 |                1 |
+----------------+---------+-------------+------------+------------------+
| Notebook       |      10 | Books       |          2 |                2 |
+----------------+---------+-------------+------------+------------------+
| Laptop         |    1200 | Electronics |          1 |                1 |
+----------------+---------+-------------+------------+------------------+
| Smartphone     |     800 | Electronics |          2 |                2 |
+----------------+---------+-------------+------------+------------------+
| Tablet         |     300 | Electronics |          3 |                3 |
+----------------+---------+-------------+------------+------------------+
| Pen            |       

## 3. LAG() and LEAD() Functions

**Definition:** Access data from previous (`LAG()`) or subsequent (`LEAD()`) rows within a partition, based on the specified `ORDER BY` clause.

**Use Case:**

*   Calculating differences or changes over time (e.g., month-over-month sales growth, year-over-year changes).
*   Comparing a current value to a previous or future value.
*   Analyzing time series data.

In [ ]:
execute_sql("""
    CREATE TABLE IF NOT EXISTS Daily_Sales (
        order_date DATE PRIMARY KEY,
        revenue DECIMAL(10, 2)
    );
""")

execute_sql("""
    INSERT INTO Daily_Sales (order_date, revenue) VALUES
    ('2023-11-20', 100.00),
    ('2023-11-21', 150.00),
    ('2023-11-22', 120.00),
    ('2023-11-23', 200.00);
""")

SQL statement executed successfully:

    CREATE TABLE IF NOT EXISTS Daily_Sales (
        order_date DATE PRIMARY KEY,
        revenue DECIMAL(10, 2)
    );

SQL statement executed successfully:

    INSERT INTO Daily_Sales (order_date, revenue) VALUES
    ('2023-11-20', 100.00),
    ('2023-11-21', 150.00),
    ('2023-11-22', 120.00),
    ('2023-11-23', 200.00);



In [31]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text(
                """
                SELECT
                    order_date,
                    revenue,
                    LAG(revenue, 1, 0) OVER (ORDER BY order_date) AS previous_day_revenue,
                    revenue - LAG(revenue, 1, 0) OVER (ORDER BY order_date) AS revenue_difference
                FROM Daily_Sales;
                """
        ))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)

except Exception as e:
    print("Error querying table:", e)

+--------------+-----------+------------------------+----------------------+
| order_date   |   revenue |   previous_day_revenue |   revenue_difference |
+==============+===========+========================+======================+
| 2023-11-20   |       100 |                      0 |                  100 |
+--------------+-----------+------------------------+----------------------+
| 2023-11-21   |       150 |                    100 |                   50 |
+--------------+-----------+------------------------+----------------------+
| 2023-11-22   |       120 |                    150 |                  -30 |
+--------------+-----------+------------------------+----------------------+
| 2023-11-23   |       200 |                    120 |                   80 |
+--------------+-----------+------------------------+----------------------+


## 4. NTILE(n) Function

**Definition:** Divides the rows in a partition into `n` groups (tiles) and assigns a tile number to each row.

**Use Case:**

*   Dividing customers into quartiles (NTILE(4)) based on spending.
*   Segmenting data into percentiles.
*   Creating equal-sized groups for analysis.

In [32]:
execute_sql("""
    CREATE TABLE IF NOT EXISTS Customer_Spending (
        customer_id SERIAL PRIMARY KEY,
        total_spent DECIMAL(10, 2)
    );
""")

SQL statement executed successfully:

    CREATE TABLE IF NOT EXISTS Customer_Spending (
        customer_id SERIAL PRIMARY KEY,
        total_spent DECIMAL(10, 2)
    );



In [33]:
execute_sql("""
    INSERT INTO Customer_Spending (total_spent) VALUES
    (100.00),
    (250.00),
    (50.00),
    (500.00),
    (120.00);
""")

SQL statement executed successfully:

    INSERT INTO Customer_Spending (total_spent) VALUES
    (100.00),
    (250.00),
    (50.00),
    (500.00),
    (120.00);



In [35]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text(
                """
                SELECT
                    customer_id,
                    total_spent,
                    NTILE(4) OVER (ORDER BY total_spent DESC) AS quartile
                FROM Customer_Spending;
                """
        ))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)

except Exception as e:
    print("Error querying table:", e)

+---------------+---------------+------------+
|   customer_id |   total_spent |   quartile |
+===============+===============+============+
|             4 |           500 |          1 |
+---------------+---------------+------------+
|             2 |           250 |          1 |
+---------------+---------------+------------+
|             5 |           120 |          2 |
+---------------+---------------+------------+
|             1 |           100 |          3 |
+---------------+---------------+------------+
|             3 |            50 |          4 |
+---------------+---------------+------------+


## 5. Aggregate Functions as Window Functions

**Definition:** Use aggregate functions (like `SUM()`, `AVG()`, `MIN()`, `MAX()`, `COUNT()`) as window functions to calculate running totals, moving averages, or other aggregate values within a defined window.

**Use Case:**

*   Trend analysis (e.g., calculating a 7-day moving average of website traffic).
*   Calculating cumulative sums (e.g., tracking total sales over time).
*   Identifying rolling minimum or maximum values.

In [36]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text(
                """
                SELECT
                    order_date,
                    revenue,
                    SUM(revenue) OVER (ORDER BY order_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_revenue
                FROM Daily_Sales;
                """
        ))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)

except Exception as e:
    print("Error querying table:", e)

+--------------+-----------+----------------------+
| order_date   |   revenue |   cumulative_revenue |
+==============+===========+======================+
| 2023-11-20   |       100 |                  100 |
+--------------+-----------+----------------------+
| 2023-11-21   |       150 |                  250 |
+--------------+-----------+----------------------+
| 2023-11-22   |       120 |                  370 |
+--------------+-----------+----------------------+
| 2023-11-23   |       200 |                  570 |
+--------------+-----------+----------------------+
